In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from bayes_opt import BayesianOptimization

import matplotlib.pyplot as plt
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

In [2]:
all_train_data = pd.read_csv('train.csv')
all_test_data = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')
all_train_data.dropna(inplace=True)
all_test_data.dropna(inplace=True)

dropped_columns = ['year_graduated']
y_train = all_train_data['Best Performance']
X_train = all_train_data.drop('Best Performance',axis=1)

#read x_test
for col in dropped_columns:
    X_train = X_train.drop(col,axis=1)
X_test = all_test_data

for col in dropped_columns:
    X_test = X_test.drop(col,axis=1)
X_train.isna().sum()

job_level                                0
job_duration_in_current_job_level        0
person_level                             0
job_duration_in_current_person_level     0
job_duration_in_current_branch           0
Employee_type                            0
gender                                   0
age                                      0
marital_status_maried(Y/N)               0
number_of_dependences                    0
Education_level                          0
GPA                                      0
job_duration_from_training               0
branch_rotation                          0
job_rotation                             0
assign_of_otherposition                  0
annual leave                             0
sick_leaves                              0
Last_achievement_%                       0
Achievement_above_100%_during3quartal    0
dtype: int64

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11152 entries, 0 to 11152
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_level                              11152 non-null  object 
 1   job_duration_in_current_job_level      11152 non-null  float64
 2   person_level                           11152 non-null  object 
 3   job_duration_in_current_person_level   11152 non-null  float64
 4   job_duration_in_current_branch         11152 non-null  float64
 5   Employee_type                          11152 non-null  object 
 6   gender                                 11152 non-null  int64  
 7   age                                    11152 non-null  int64  
 8   marital_status_maried(Y/N)             11152 non-null  object 
 9   number_of_dependences                  11152 non-null  int64  
 10  Education_level                        11152 non-null  object 
 11  GP

## Correct data types based on its value

In [4]:
tahun = 2020
X_train['age'] = tahun - X_train['age']
X_test['age'] = tahun - X_test['age']


In [5]:
X_train.head(9)

,job_level,job_duration_in_current_job_level,person_level,job_duration_in_current_person_level,job_duration_in_current_branch,Employee_type,gender,age,marital_status_maried(Y/N),number_of_dependences,Education_level,GPA,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,Achievement_above_100%_during3quartal
0,JG04,1.352775,PG03,1.352775,1.732051,RM_type_A,2,35,Y,1,level_0,2.86,4,2,3,2,3,0,46.37,0.0
1,JG04,1.292285,PG03,1.292285,1.039230,RM_type_A,2,31,Y,0,level_1,7.28,4,4,3,0,0,1,47.68,0.0
2,JG05,2.565151,PG06,2.308679,1.780449,RM_type_C,1,50,Y,1,level_1,0.00,28,10,9,6,2,1,40.73,0.0
3,JG05,2.828427,PG06,1.000000,1.385641,RM_type_A,2,53,Y,2,level_1,0.00,29,11,6,0,4,10,47.42,0.0
4,JG05,2.828427,PG06,2.828427,0.707107,RM_type_A,2,55,Y,3,level_1,7.20,30,9,8,2,3,0,47.18,0.0
5,JG05,2.828427,PG06,2.121320,1.118034,RM_type_A,1,49,Y,3,level_1,0.00,28,7,9,4,0,0,48.46,0.0
6,JG05,2.828427,PG06,2.345208,2.102380,RM_type_A,1,51,Y,2,level_1,0.00,29,8,6,2,8,0,60.38,0.0
7,JG05,2.828427,PG06,2.236068,1.581139,RM_type_C,2,52,Y,2,level_1,0.00,29,6,8,1,3,4,14.71,0.0
8,JG04,1.118034,PG03,1.118034,0.412311,RM_type_B,1,31,Y,0,level_1,3.01,3,2,2,0,9,0,26.76,0.0


In [6]:
cat_col=[col for col in X_train.columns if X_train[col].dtype=='O']
cat_col_test=[col for col in X_test.columns if X_test[col].dtype=='O']
categorical=X_train[cat_col]
categorical_test=X_test[cat_col_test]
categorical.head()

,job_level,person_level,Employee_type,marital_status_maried(Y/N),Education_level
0,JG04,PG03,RM_type_A,Y,level_0
1,JG04,PG03,RM_type_A,Y,level_1
2,JG05,PG06,RM_type_C,Y,level_1
3,JG05,PG06,RM_type_A,Y,level_1
4,JG05,PG06,RM_type_A,Y,level_1


In [7]:
#now extract how many categories in each cat_feature
for feature in categorical.columns:
    print('{} has total {} categories \n'.format(feature,len(categorical[feature].value_counts())))
print('---------------------------')
for feature in categorical_test.columns:
    print('{} has total {} categories \n'.format(feature,len(categorical_test[feature].value_counts())))

job_level has total 3 categories 

person_level has total 8 categories 

Employee_type has total 3 categories 

marital_status_maried(Y/N) has total 2 categories 

Education_level has total 6 categories 

---------------------------
job_level has total 4 categories 

person_level has total 7 categories 

Employee_type has total 3 categories 

marital_status_maried(Y/N) has total 2 categories 

Education_level has total 6 categories 



In [8]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
for i in ['job_level', 'person_level', 'Employee_type', 'marital_status_maried(Y/N)','Education_level']:
    categorical[i]=encoder.fit_transform(categorical[i])
    categorical_test[i]=encoder.fit_transform(categorical_test[i])

C:\Users\farha\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\farha\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
# data contain values
cont_col=[col for col in X_train.columns if X_train[col].dtype!='O']
cont_col_test=[col for col in X_test.columns if X_test[col].dtype!='O']
cont_col

['job_duration_in_current_job_level',
 'job_duration_in_current_person_level',
 'job_duration_in_current_branch',
 'gender',
 'age',
 'number_of_dependences',
 'GPA',
 'job_duration_from_training',
 'branch_rotation',
 'job_rotation',
 'assign_of_otherposition',
 'annual leave',
 'sick_leaves',
 'Last_achievement_%',
 'Achievement_above_100%_during3quartal']

In [10]:

X_train_full=pd.concat([categorical,X_train[cont_col]],axis=1)
X_test_full=pd.concat([categorical_test,X_test[cont_col_test]],axis=1)

In [11]:
y_train

0        0
1        0
2        0
3        1
4        0
        ..
11148    0
11149    0
11150    0
11151    0
11152    0
Name: Best Performance, Length: 11152, dtype: int64

In [12]:
X_train_full

,job_level,person_level,Employee_type,marital_status_maried(Y/N),Education_level,job_duration_in_current_job_level,job_duration_in_current_person_level,job_duration_in_current_branch,gender,age,number_of_dependences,GPA,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,Achievement_above_100%_during3quartal
0,1,2,0,1,0,1.352775,1.352775,1.732051,2,35,1,2.86,4,2,3,2,3,0,46.37,0.0
1,1,2,0,1,1,1.292285,1.292285,1.039230,2,31,0,7.28,4,4,3,0,0,1,47.68,0.0
2,2,5,2,1,1,2.565151,2.308679,1.780449,1,50,1,0.00,28,10,9,6,2,1,40.73,0.0
3,2,5,0,1,1,2.828427,1.000000,1.385641,2,53,2,0.00,29,11,6,0,4,10,47.42,0.0
4,2,5,0,1,1,2.828427,2.828427,0.707107,2,55,3,7.20,30,9,8,2,3,0,47.18,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11148,1,2,0,1,5,1.256981,1.256981,1.224745,2,34,1,3.33,4,3,3,0,4,1,50.87,0.0
11149,1,2,1,1,5,1.385641,1.385641,1.802776,1,32,2,3.00,3,1,2,0,6,0,79.91,0.0
11150,1,2,1,1,5,1.256981,1.256981,1.385641,2,29,0,3.48,2,1,2,0,4,0,78.38,0.0
11151,1,2,0,1,5,1.118034,1.118034,0.000000,1,33,1,3.53,5,5,4,0,4,1,85.94,1.0


In [13]:
X_train_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11152 entries, 0 to 11152
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_level                              11152 non-null  int32  
 1   person_level                           11152 non-null  int32  
 2   Employee_type                          11152 non-null  int32  
 3   marital_status_maried(Y/N)             11152 non-null  int32  
 4   Education_level                        11152 non-null  int32  
 5   job_duration_in_current_job_level      11152 non-null  float64
 6   job_duration_in_current_person_level   11152 non-null  float64
 7   job_duration_in_current_branch         11152 non-null  float64
 8   gender                                 11152 non-null  int64  
 9   age                                    11152 non-null  int64  
 10  number_of_dependences                  11152 non-null  int64  
 11  GP

In [14]:
X_train_full['Achievement_above_100%_during3quartal'] = pd.DataFrame(X_train_full['Achievement_above_100%_during3quartal'].astype('int32'))
X_test_full['Achievement_above_100%_during3quartal'] = pd.DataFrame(X_test_full['Achievement_above_100%_during3quartal'].astype('int32'))



In [15]:

X_train_full[['age','job_duration_from_training','branch_rotation','job_rotation','assign_of_otherposition','annual leave','sick_leaves']] = pd.DataFrame(X_train_full[['age','job_duration_from_training','branch_rotation','job_rotation','assign_of_otherposition','annual leave','sick_leaves']].astype('float64'))

X_test_full[['age','job_duration_from_training','branch_rotation','job_rotation','assign_of_otherposition','annual leave','sick_leaves']] = pd.DataFrame(X_test_full[['age','job_duration_from_training','branch_rotation','job_rotation','assign_of_otherposition','annual leave','sick_leaves']].astype('float64'))

In [16]:
X_train_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11152 entries, 0 to 11152
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_level                              11152 non-null  int32  
 1   person_level                           11152 non-null  int32  
 2   Employee_type                          11152 non-null  int32  
 3   marital_status_maried(Y/N)             11152 non-null  int32  
 4   Education_level                        11152 non-null  int32  
 5   job_duration_in_current_job_level      11152 non-null  float64
 6   job_duration_in_current_person_level   11152 non-null  float64
 7   job_duration_in_current_branch         11152 non-null  float64
 8   gender                                 11152 non-null  int64  
 9   age                                    11152 non-null  float64
 10  number_of_dependences                  11152 non-null  int64  
 11  GP

In [17]:
float_col=[col for col in X_train_full.columns if X_train_full[col].dtype=='float64']
float_col_test=[col for col in X_test_full.columns if X_test_full[col].dtype=='float64']

In [18]:
X_test_full[float_col]

,job_duration_in_current_job_level,job_duration_in_current_person_level,job_duration_in_current_branch,age,GPA,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%
0,1.256981,1.256981,0.707107,32.0,3.08,6.0,4.0,2.0,6.0,3.0,7.0,84.02
1,1.224745,1.224745,1.256981,29.0,3.31,2.0,2.0,2.0,0.0,1.0,0.0,63.74
2,0.500000,0.500000,1.081665,31.0,3.44,4.0,2.0,2.0,0.0,2.0,1.0,88.19
3,1.256981,1.256981,1.802776,30.0,0.00,3.0,1.0,3.0,0.0,3.0,0.0,25.94
4,1.352775,1.352775,1.224745,35.0,3.34,5.0,3.0,2.0,0.0,3.0,0.0,82.32
...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1.292285,1.292285,0.707107,32.0,3.01,2.0,2.0,2.0,0.0,4.0,0.0,58.60
5996,1.352775,1.352775,1.581139,31.0,3.56,5.0,3.0,2.0,7.0,3.0,0.0,90.64
5997,1.385641,1.385641,1.224745,29.0,3.59,3.0,2.0,3.0,0.0,16.0,0.0,13.02
5998,1.385641,1.385641,0.648074,40.0,2.93,14.0,7.0,6.0,8.0,2.0,0.0,82.26


In [19]:
scaler = MinMaxScaler()
X_float_scale = scaler.fit_transform(X_train_full[float_col])
X_float_scale_test = scaler.fit_transform(X_test_full[float_col_test])

In [20]:
X_scale_train = pd.DataFrame()
X_scale_train[float_col] = pd.DataFrame(X_float_scale)
X_scale_test = pd.DataFrame()
X_scale_test[float_col_test] = pd.DataFrame(X_float_scale_test)

In [21]:
X_scale_test

,job_duration_in_current_job_level,job_duration_in_current_person_level,job_duration_in_current_branch,age,GPA,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%
0,0.431141,0.444410,0.258199,0.21875,0.008084,0.121212,0.1875,0.076923,0.222222,0.142857,0.060870,0.614747
1,0.420084,0.433013,0.458984,0.12500,0.008688,0.000000,0.0625,0.076923,0.000000,0.047619,0.000000,0.444826
2,0.171499,0.176777,0.394968,0.18750,0.009029,0.060606,0.0625,0.076923,0.000000,0.095238,0.008696,0.649686
3,0.431141,0.444410,0.658281,0.15625,0.000000,0.030303,0.0000,0.153846,0.000000,0.142857,0.000000,0.128111
4,0.463998,0.478278,0.447214,0.31250,0.008766,0.090909,0.1250,0.076923,0.000000,0.142857,0.000000,0.600503
...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.443250,0.456892,0.258199,0.21875,0.007900,0.000000,0.0625,0.076923,0.000000,0.190476,0.000000,0.401760
5996,0.463998,0.478278,0.577350,0.18750,0.009344,0.090909,0.1250,0.076923,0.259259,0.142857,0.000000,0.670214
5997,0.475271,0.489898,0.447214,0.12500,0.009423,0.030303,0.0625,0.153846,0.000000,0.761905,0.000000,0.019858
5998,0.475271,0.489898,0.236643,0.46875,0.007690,0.363636,0.3750,0.384615,0.296296,0.095238,0.000000,0.600000


In [22]:
int_col=[col for col in X_train_full.columns if X_train_full[col].dtype!='float64']
int_col_test=[col for col in X_test_full.columns if X_test_full[col].dtype!='float64']

In [23]:
X_train_full[int_col].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11152 entries, 0 to 11152
Data columns (total 8 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   job_level                              11152 non-null  int32
 1   person_level                           11152 non-null  int32
 2   Employee_type                          11152 non-null  int32
 3   marital_status_maried(Y/N)             11152 non-null  int32
 4   Education_level                        11152 non-null  int32
 5   gender                                 11152 non-null  int64
 6   number_of_dependences                  11152 non-null  int64
 7   Achievement_above_100%_during3quartal  11152 non-null  int32
dtypes: int32(6), int64(2)
memory usage: 522.8 KB


In [24]:

X_train=pd.concat([X_scale_train,X_train_full[int_col]],axis=1,join='inner')
X_test=pd.concat([X_scale_test,X_test_full[int_col_test]],axis=1,join='inner')
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11151 entries, 0 to 11151
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_duration_in_current_job_level      11151 non-null  float64
 1   job_duration_in_current_person_level   11151 non-null  float64
 2   job_duration_in_current_branch         11151 non-null  float64
 3   age                                    11151 non-null  float64
 4   GPA                                    11151 non-null  float64
 5   job_duration_from_training             11151 non-null  float64
 6   branch_rotation                        11151 non-null  float64
 7   job_rotation                           11151 non-null  float64
 8   assign_of_otherposition                11151 non-null  float64
 9   annual leave                           11151 non-null  float64
 10  sick_leaves                            11151 non-null  float64
 11  La

In [25]:
y_train_full = y_train[:-1] 

In [26]:
X_test

,job_duration_in_current_job_level,job_duration_in_current_person_level,job_duration_in_current_branch,age,GPA,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,job_level,person_level,Employee_type,marital_status_maried(Y/N),Education_level,gender,number_of_dependences,Achievement_above_100%_during3quartal
0,0.431141,0.444410,0.258199,0.21875,0.008084,0.121212,0.1875,0.076923,0.222222,0.142857,0.060870,0.614747,1,2,0,0,3,1,0,0
1,0.420084,0.433013,0.458984,0.12500,0.008688,0.000000,0.0625,0.076923,0.000000,0.047619,0.000000,0.444826,1,2,1,0,4,2,0,0
2,0.171499,0.176777,0.394968,0.18750,0.009029,0.060606,0.0625,0.076923,0.000000,0.095238,0.008696,0.649686,1,2,0,1,4,2,1,1
3,0.431141,0.444410,0.658281,0.15625,0.000000,0.030303,0.0000,0.153846,0.000000,0.142857,0.000000,0.128111,1,2,0,1,4,2,1,0
4,0.463998,0.478278,0.447214,0.31250,0.008766,0.090909,0.1250,0.076923,0.000000,0.142857,0.000000,0.600503,1,2,1,1,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.443250,0.456892,0.258199,0.21875,0.007900,0.000000,0.0625,0.076923,0.000000,0.190476,0.000000,0.401760,1,2,1,1,4,2,0,0
5996,0.463998,0.478278,0.577350,0.18750,0.009344,0.090909,0.1250,0.076923,0.259259,0.142857,0.000000,0.670214,1,2,0,1,4,2,1,3
5997,0.475271,0.489898,0.447214,0.12500,0.009423,0.030303,0.0625,0.153846,0.000000,0.761905,0.000000,0.019858,1,2,0,1,4,2,0,0
5998,0.475271,0.489898,0.236643,0.46875,0.007690,0.363636,0.3750,0.384615,0.296296,0.095238,0.000000,0.600000,2,4,0,1,4,2,2,0


## Split

In [27]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train,y_train_full, test_size=0.2, random_state=38)
# X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train_full,y_train, test_size=0.2, random_state=38)



In [28]:

from catboost import Pool, CatBoostClassifier, cv
model = CatBoostClassifier(iterations=1500, learning_rate=0.01, l2_leaf_reg=4, depth=8, rsm=0.98, loss_function= 'Logloss', eval_metric='AUC',use_best_model=True,random_seed=42)

In [29]:
cate_features_index = np.where(X_train_.dtypes != float)[0] 


In [30]:
model.fit(X_train_, y_train_, cat_features=cate_features_index,eval_set=(X_test_, y_test_))


0:	test: 0.5131166	best: 0.5131166 (0)	total: 94.1ms	remaining: 2m 21s
1:	test: 0.5167497	best: 0.5167497 (1)	total: 130ms	remaining: 1m 37s
2:	test: 0.5266821	best: 0.5266821 (2)	total: 169ms	remaining: 1m 24s
3:	test: 0.5251643	best: 0.5266821 (2)	total: 206ms	remaining: 1m 16s
4:	test: 0.5392711	best: 0.5392711 (4)	total: 243ms	remaining: 1m 12s
5:	test: 0.5439016	best: 0.5439016 (5)	total: 275ms	remaining: 1m 8s
6:	test: 0.5477596	best: 0.5477596 (6)	total: 334ms	remaining: 1m 11s
7:	test: 0.5567918	best: 0.5567918 (7)	total: 371ms	remaining: 1m 9s
8:	test: 0.5601695	best: 0.5601695 (8)	total: 399ms	remaining: 1m 6s
9:	test: 0.5492782	best: 0.5601695 (8)	total: 439ms	remaining: 1m 5s
10:	test: 0.5515710	best: 0.5601695 (8)	total: 453ms	remaining: 1m 1s
11:	test: 0.5473797	best: 0.5601695 (8)	total: 496ms	remaining: 1m 1s
12:	test: 0.5430937	best: 0.5601695 (8)	total: 542ms	remaining: 1m 1s
13:	test: 0.5437956	best: 0.5601695 (8)	total: 601ms	remaining: 1m 3s
14:	test: 0.5481595	bes

121:	test: 0.5379436	best: 0.5601695 (8)	total: 4.16s	remaining: 47s
122:	test: 0.5379436	best: 0.5601695 (8)	total: 4.17s	remaining: 46.7s
123:	test: 0.5379436	best: 0.5601695 (8)	total: 4.17s	remaining: 46.3s
124:	test: 0.5377975	best: 0.5601695 (8)	total: 4.22s	remaining: 46.4s
125:	test: 0.5363327	best: 0.5601695 (8)	total: 4.24s	remaining: 46.3s
126:	test: 0.5359825	best: 0.5601695 (8)	total: 4.29s	remaining: 46.3s
127:	test: 0.5359825	best: 0.5601695 (8)	total: 4.3s	remaining: 46.1s
128:	test: 0.5360162	best: 0.5601695 (8)	total: 4.33s	remaining: 46s
129:	test: 0.5376850	best: 0.5601695 (8)	total: 4.37s	remaining: 46s
130:	test: 0.5370442	best: 0.5601695 (8)	total: 4.39s	remaining: 45.9s
131:	test: 0.5365575	best: 0.5601695 (8)	total: 4.4s	remaining: 45.6s
132:	test: 0.5369093	best: 0.5601695 (8)	total: 4.42s	remaining: 45.5s
133:	test: 0.5365350	best: 0.5601695 (8)	total: 4.46s	remaining: 45.5s
134:	test: 0.5374056	best: 0.5601695 (8)	total: 4.5s	remaining: 45.5s
135:	test: 0.53

242:	test: 0.5454877	best: 0.5601695 (8)	total: 7.58s	remaining: 39.2s
243:	test: 0.5454877	best: 0.5601695 (8)	total: 7.59s	remaining: 39.1s
244:	test: 0.5453817	best: 0.5601695 (8)	total: 7.62s	remaining: 39.1s
245:	test: 0.5454893	best: 0.5601695 (8)	total: 7.65s	remaining: 39s
246:	test: 0.5455021	best: 0.5601695 (8)	total: 7.68s	remaining: 39s
247:	test: 0.5452532	best: 0.5601695 (8)	total: 7.72s	remaining: 39s
248:	test: 0.5452532	best: 0.5601695 (8)	total: 7.72s	remaining: 38.8s
249:	test: 0.5449544	best: 0.5601695 (8)	total: 7.76s	remaining: 38.8s
250:	test: 0.5448870	best: 0.5601695 (8)	total: 7.8s	remaining: 38.8s
251:	test: 0.5447617	best: 0.5601695 (8)	total: 7.85s	remaining: 38.9s
252:	test: 0.5447617	best: 0.5601695 (8)	total: 7.86s	remaining: 38.8s
253:	test: 0.5439875	best: 0.5601695 (8)	total: 7.9s	remaining: 38.8s
254:	test: 0.5437322	best: 0.5601695 (8)	total: 7.94s	remaining: 38.8s
255:	test: 0.5437691	best: 0.5601695 (8)	total: 7.99s	remaining: 38.8s
256:	test: 0.5

364:	test: 0.5412298	best: 0.5601695 (8)	total: 11.8s	remaining: 36.7s
365:	test: 0.5415414	best: 0.5601695 (8)	total: 11.9s	remaining: 36.7s
366:	test: 0.5415350	best: 0.5601695 (8)	total: 11.9s	remaining: 36.7s
367:	test: 0.5419060	best: 0.5601695 (8)	total: 11.9s	remaining: 36.7s
368:	test: 0.5419060	best: 0.5601695 (8)	total: 12s	remaining: 36.7s
369:	test: 0.5426657	best: 0.5601695 (8)	total: 12s	remaining: 36.7s
370:	test: 0.5425324	best: 0.5601695 (8)	total: 12s	remaining: 36.6s
371:	test: 0.5424954	best: 0.5601695 (8)	total: 12.1s	remaining: 36.6s
372:	test: 0.5426336	best: 0.5601695 (8)	total: 12.1s	remaining: 36.6s
373:	test: 0.5426223	best: 0.5601695 (8)	total: 12.2s	remaining: 36.6s
374:	test: 0.5423702	best: 0.5601695 (8)	total: 12.2s	remaining: 36.6s
375:	test: 0.5421855	best: 0.5601695 (8)	total: 12.3s	remaining: 36.6s
376:	test: 0.5422754	best: 0.5601695 (8)	total: 12.3s	remaining: 36.6s
377:	test: 0.5418899	best: 0.5601695 (8)	total: 12.3s	remaining: 36.6s
378:	test: 0

485:	test: 0.5415285	best: 0.5601695 (8)	total: 16.3s	remaining: 34s
486:	test: 0.5417100	best: 0.5601695 (8)	total: 16.4s	remaining: 34s
487:	test: 0.5417100	best: 0.5601695 (8)	total: 16.4s	remaining: 34s
488:	test: 0.5415671	best: 0.5601695 (8)	total: 16.4s	remaining: 33.9s
489:	test: 0.5418128	best: 0.5601695 (8)	total: 16.4s	remaining: 33.9s
490:	test: 0.5417197	best: 0.5601695 (8)	total: 16.5s	remaining: 33.9s
491:	test: 0.5418578	best: 0.5601695 (8)	total: 16.5s	remaining: 33.8s
492:	test: 0.5419429	best: 0.5601695 (8)	total: 16.5s	remaining: 33.8s
493:	test: 0.5419526	best: 0.5601695 (8)	total: 16.6s	remaining: 33.8s
494:	test: 0.5420361	best: 0.5601695 (8)	total: 16.6s	remaining: 33.8s
495:	test: 0.5422352	best: 0.5601695 (8)	total: 16.7s	remaining: 33.8s
496:	test: 0.5424456	best: 0.5601695 (8)	total: 16.7s	remaining: 33.8s
497:	test: 0.5422818	best: 0.5601695 (8)	total: 16.8s	remaining: 33.8s
498:	test: 0.5419943	best: 0.5601695 (8)	total: 16.8s	remaining: 33.8s
499:	test: 0

...

956:	test: 0.5446461	best: 0.5601695 (8)	total: 34.1s	remaining: 19.3s
957:	test: 0.5446766	best: 0.5601695 (8)	total: 34.1s	remaining: 19.3s
958:	test: 0.5446429	best: 0.5601695 (8)	total: 34.1s	remaining: 19.3s
959:	test: 0.5447457	best: 0.5601695 (8)	total: 34.2s	remaining: 19.2s
960:	test: 0.5447457	best: 0.5601695 (8)	total: 34.2s	remaining: 19.2s
961:	test: 0.5447890	best: 0.5601695 (8)	total: 34.2s	remaining: 19.1s
962:	test: 0.5447778	best: 0.5601695 (8)	total: 34.2s	remaining: 19.1s
963:	test: 0.5450476	best: 0.5601695 (8)	total: 34.3s	remaining: 19.1s
964:	test: 0.5450267	best: 0.5601695 (8)	total: 34.3s	remaining: 19s
965:	test: 0.5452243	best: 0.5601695 (8)	total: 34.4s	remaining: 19s
966:	test: 0.5449079	best: 0.5601695 (8)	total: 34.4s	remaining: 19s
967:	test: 0.5449143	best: 0.5601695 (8)	total: 34.4s	remaining: 18.9s
968:	test: 0.5449721	best: 0.5601695 (8)	total: 34.5s	remaining: 18.9s
969:	test: 0.5449223	best: 0.5601695 (8)	total: 34.5s	remaining: 18.8s
970:	test: 0

1077:	test: 0.5454861	best: 0.5601695 (8)	total: 38.5s	remaining: 15.1s
1078:	test: 0.5456676	best: 0.5601695 (8)	total: 38.6s	remaining: 15.1s
1079:	test: 0.5453592	best: 0.5601695 (8)	total: 38.6s	remaining: 15s
1080:	test: 0.5454957	best: 0.5601695 (8)	total: 38.7s	remaining: 15s
1081:	test: 0.5455760	best: 0.5601695 (8)	total: 38.7s	remaining: 14.9s
1082:	test: 0.5455760	best: 0.5601695 (8)	total: 38.7s	remaining: 14.9s
1083:	test: 0.5455198	best: 0.5601695 (8)	total: 38.8s	remaining: 14.9s
1084:	test: 0.5455198	best: 0.5601695 (8)	total: 38.8s	remaining: 14.8s
1085:	test: 0.5455021	best: 0.5601695 (8)	total: 38.8s	remaining: 14.8s
1086:	test: 0.5453255	best: 0.5601695 (8)	total: 38.9s	remaining: 14.8s
1087:	test: 0.5453608	best: 0.5601695 (8)	total: 38.9s	remaining: 14.7s
1088:	test: 0.5452211	best: 0.5601695 (8)	total: 38.9s	remaining: 14.7s
1089:	test: 0.5452580	best: 0.5601695 (8)	total: 39s	remaining: 14.7s
1090:	test: 0.5457832	best: 0.5601695 (8)	total: 39s	remaining: 14.6s


1194:	test: 0.5428777	best: 0.5601695 (8)	total: 43.1s	remaining: 11s
1195:	test: 0.5434720	best: 0.5601695 (8)	total: 43.1s	remaining: 11s
1196:	test: 0.5437338	best: 0.5601695 (8)	total: 43.2s	remaining: 10.9s
1197:	test: 0.5437771	best: 0.5601695 (8)	total: 43.2s	remaining: 10.9s
1198:	test: 0.5437771	best: 0.5601695 (8)	total: 43.2s	remaining: 10.9s
1199:	test: 0.5437257	best: 0.5601695 (8)	total: 43.3s	remaining: 10.8s
1200:	test: 0.5439795	best: 0.5601695 (8)	total: 43.3s	remaining: 10.8s
1201:	test: 0.5439105	best: 0.5601695 (8)	total: 43.4s	remaining: 10.8s
1202:	test: 0.5438880	best: 0.5601695 (8)	total: 43.4s	remaining: 10.7s
1203:	test: 0.5439410	best: 0.5601695 (8)	total: 43.4s	remaining: 10.7s
1204:	test: 0.5439265	best: 0.5601695 (8)	total: 43.5s	remaining: 10.6s
1205:	test: 0.5440277	best: 0.5601695 (8)	total: 43.5s	remaining: 10.6s
1206:	test: 0.5440261	best: 0.5601695 (8)	total: 43.6s	remaining: 10.6s
1207:	test: 0.5440791	best: 0.5601695 (8)	total: 43.6s	remaining: 10

1311:	test: 0.5470553	best: 0.5601695 (8)	total: 47.8s	remaining: 6.85s
1312:	test: 0.5472416	best: 0.5601695 (8)	total: 47.8s	remaining: 6.81s
1313:	test: 0.5473508	best: 0.5601695 (8)	total: 47.9s	remaining: 6.78s
1314:	test: 0.5473235	best: 0.5601695 (8)	total: 47.9s	remaining: 6.74s
1315:	test: 0.5473685	best: 0.5601695 (8)	total: 47.9s	remaining: 6.7s
1316:	test: 0.5474424	best: 0.5601695 (8)	total: 48s	remaining: 6.67s
1317:	test: 0.5473267	best: 0.5601695 (8)	total: 48s	remaining: 6.63s
1318:	test: 0.5473348	best: 0.5601695 (8)	total: 48.1s	remaining: 6.6s
1319:	test: 0.5474038	best: 0.5601695 (8)	total: 48.1s	remaining: 6.56s
1320:	test: 0.5477636	best: 0.5601695 (8)	total: 48.2s	remaining: 6.52s
1321:	test: 0.5480013	best: 0.5601695 (8)	total: 48.2s	remaining: 6.49s
1322:	test: 0.5476223	best: 0.5601695 (8)	total: 48.2s	remaining: 6.45s
1323:	test: 0.5475339	best: 0.5601695 (8)	total: 48.3s	remaining: 6.42s
1324:	test: 0.5475387	best: 0.5601695 (8)	total: 48.3s	remaining: 6.38

1430:	test: 0.5496605	best: 0.5601695 (8)	total: 52.7s	remaining: 2.54s
1431:	test: 0.5497022	best: 0.5601695 (8)	total: 52.7s	remaining: 2.5s
1432:	test: 0.5499158	best: 0.5601695 (8)	total: 52.8s	remaining: 2.47s
1433:	test: 0.5497841	best: 0.5601695 (8)	total: 52.8s	remaining: 2.43s
1434:	test: 0.5497472	best: 0.5601695 (8)	total: 52.8s	remaining: 2.39s
1435:	test: 0.5498580	best: 0.5601695 (8)	total: 52.9s	remaining: 2.36s
1436:	test: 0.5501712	best: 0.5601695 (8)	total: 52.9s	remaining: 2.32s
1437:	test: 0.5504218	best: 0.5601695 (8)	total: 53s	remaining: 2.28s
1438:	test: 0.5503559	best: 0.5601695 (8)	total: 53s	remaining: 2.25s
1439:	test: 0.5503222	best: 0.5601695 (8)	total: 53s	remaining: 2.21s
1440:	test: 0.5504186	best: 0.5601695 (8)	total: 53.1s	remaining: 2.17s
1441:	test: 0.5503029	best: 0.5601695 (8)	total: 53.1s	remaining: 2.14s
1442:	test: 0.5503784	best: 0.5601695 (8)	total: 53.2s	remaining: 2.1s
1443:	test: 0.5504410	best: 0.5601695 (8)	total: 53.2s	remaining: 2.06s


In [35]:
X_test

,job_duration_in_current_job_level,job_duration_in_current_person_level,job_duration_in_current_branch,age,GPA,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,job_level,person_level,Employee_type,marital_status_maried(Y/N),Education_level,gender,number_of_dependences,Achievement_above_100%_during3quartal
0,0.431141,0.444410,0.258199,0.21875,0.008084,0.121212,0.1875,0.076923,0.222222,0.142857,0.060870,0.614747,1,2,0,0,3,1,0,0
1,0.420084,0.433013,0.458984,0.12500,0.008688,0.000000,0.0625,0.076923,0.000000,0.047619,0.000000,0.444826,1,2,1,0,4,2,0,0
2,0.171499,0.176777,0.394968,0.18750,0.009029,0.060606,0.0625,0.076923,0.000000,0.095238,0.008696,0.649686,1,2,0,1,4,2,1,1
3,0.431141,0.444410,0.658281,0.15625,0.000000,0.030303,0.0000,0.153846,0.000000,0.142857,0.000000,0.128111,1,2,0,1,4,2,1,0
4,0.463998,0.478278,0.447214,0.31250,0.008766,0.090909,0.1250,0.076923,0.000000,0.142857,0.000000,0.600503,1,2,1,1,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.443250,0.456892,0.258199,0.21875,0.007900,0.000000,0.0625,0.076923,0.000000,0.190476,0.000000,0.401760,1,2,1,1,4,2,0,0
5996,0.463998,0.478278,0.577350,0.18750,0.009344,0.090909,0.1250,0.076923,0.259259,0.142857,0.000000,0.670214,1,2,0,1,4,2,1,3
5997,0.475271,0.489898,0.447214,0.12500,0.009423,0.030303,0.0625,0.153846,0.000000,0.761905,0.000000,0.019858,1,2,0,1,4,2,0,0
5998,0.475271,0.489898,0.236643,0.46875,0.007690,0.363636,0.3750,0.384615,0.296296,0.095238,0.000000,0.600000,2,4,0,1,4,2,2,0


In [32]:
pred = model.predict_proba(X_test)
preds= pred[:,1]

In [33]:
preds

array([0.46277506, 0.46417404, 0.46165739, ..., 0.46464816, 0.46184057,
       0.46199521])

In [34]:
#generating submission csv
submission = pd.DataFrame(preds)
#save the file to your directory
submission.to_csv('test-catboost-custom.csv',index_label='index',header=['Best Performance'])